In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(project_root)



In [3]:
from coolprompt.language_model.llm import DefaultLLM

model = DefaultLLM.init(langchain_config={
    'max_new_tokens': 1000,
    "temperature": 0.0,
})


INFO 06-23 13:10:59 __init__.py:207] Automatically detected platform cuda.
WARNING 06-23 13:11:00 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 06-23 13:11:07 config.py:549] This model supports multiple tasks: {'reward', 'generate', 'classify', 'score', 'embed'}. Defaulting to 'generate'.
INFO 06-23 13:11:07 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='t-tech/T-lite-it-1.0', speculative_config=None, tokenizer='t-tech/T-lite-it-1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 06-23 13:11:17 model_runner.py:1115] Loading model weights took 14.2426 GB
INFO 06-23 13:11:20 worker.py:267] Memory profiling takes 2.42 seconds
INFO 06-23 13:11:20 worker.py:267] the current vLLM instance can use total_gpu_memory (79.15GiB) x gpu_memory_utilization (0.90) = 71.24GiB
INFO 06-23 13:11:20 worker.py:267] model weights take 14.24GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 4.35GiB; the rest of the memory reserved for KV Cache is 52.55GiB.
INFO 06-23 13:11:20 executor_base.py:111] # cuda blocks: 61497, # CPU blocks: 4681
INFO 06-23 13:11:20 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 30.03x
INFO 06-23 13:11:24 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:24<00:00,  1.40it/s]

INFO 06-23 13:11:49 model_runner.py:1562] Graph capturing finished in 25 secs, took 0.19 GiB
INFO 06-23 13:11:49 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 32.48 seconds


In [4]:
from coolprompt.assistant import PromptTuner
tuner = PromptTuner(model=model)

In [5]:
from datasets import load_dataset

sst2 = load_dataset("sst2")
n_instances = 20
dataset = sst2['train']['sentence'][:n_instances]
targets = sst2['train']['label'][:n_instances]

In [6]:
start_prompt = "Please perform Sentiment Classification task."

In [11]:
final_prompt = tuner.run(
    start_prompt=start_prompt,
    task='classification',
    dataset=dataset,
    target=targets,
    method='distill',
    use_cache=True,
    num_epochs=1
)

[2025-06-23 13:17:13,069] - Starting DistillPrompt optimization...
[2025-06-23 13:17:13,069] - Starting DistillPrompt optimization...
INFO:Distiller:Starting DistillPrompt optimization...
Processed prompts: 100%|██████████| 15/15 [00:12<00:00,  1.24it/s, est. speed input: 61.04 toks/s, output: 785.71 toks/s]
[2025-06-23 13:17:25,209] - Starting round 0
[2025-06-23 13:17:25,209] - Starting round 0
INFO:Distiller:Starting round 0

Processed prompts: 100%|██████████| 4/4 [00:00<00:00,  5.42it/s, est. speed input: 618.98 toks/s, output: 238.90 toks/s]

Processed prompts: 100%|██████████| 15/15 [00:12<00:00,  1.21it/s, est. speed input: 71.96 toks/s, output: 931.33 toks/s]

Processed prompts: 100%|██████████| 15/15 [00:00<00:00, 81.47it/s, est. speed input: 6234.32 toks/s, output: 653.31 toks/s]

Processed prompts: 100%|██████████| 15/15 [00:11<00:00,  1.27it/s, est. speed input: 120.73 toks/s, output: 402.63 toks/s]

Processed prompts: 100%|██████████| 15/15 [00:12<00:00,  1.23it/s, est. s

In [12]:
print("PROMPT:", final_prompt)

PROMPT: Perform a sentiment classification task by analyzing the sentiment of the given text as either positive, negative, or neutral. Ensure that your analysis is clear, focused, and accurate.


In [13]:
print("INITIAL METRIC:", tuner.init_metric)

INITIAL METRIC: 0.375


In [14]:
print("FINAL METRIC:", tuner.final_metric)

FINAL METRIC: 0.9466666666666667
